In [11]:
print("Hello World!")

Hello World!


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping


In [12]:

import pandas as pd

# Load ERCOT electricity load data
ercot_data = pd.read_excel(r'D:\1\Load\DataSet\LoadTexasERCOT.xlsx')

# Convert 'Hour_End' to datetime and set it as index
ercot_data['Hour_End'] = pd.to_datetime(ercot_data['Hour_End'])
ercot_data.set_index('Hour_End', inplace=True)

# Load weather data
weather_data1 = pd.read_excel(r'D:\1\Load\DataSet\WeatherStation1.xlsx')
weather_data2 = pd.read_excel(r'D:\1\Load\DataSet\WeatherStation2.xlsx')
weather_data3 = pd.read_excel(r'D:\1\Load\DataSet\WeatherStation3.xlsx')

# Convert Hour, Minute, and Date columns to datetime for weather data
for weather_data in [weather_data1, weather_data2, weather_data3]:
    weather_data['Datetime'] = pd.to_datetime(
        weather_data[['Year', 'Month', 'Day', 'Hour', 'Minute']]
    )
    weather_data.set_index('Datetime', inplace=True)

# Merge weather data with ERCOT data on timestamp
merged_data = ercot_data.copy()
for weather_data in [weather_data1, weather_data2, weather_data3]:
    merged_data = merged_data.merge(weather_data, left_index=True, right_index=True, how='outer')

# Fill missing values
merged_data.fillna(method='ffill', inplace=True)
merged_data.fillna(method='bfill', inplace=True)

# Normalize/scale features (excluding target 'ERCOT')
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
features = merged_data.drop(columns=['ERCOT'])
scaled_features = scaler.fit_transform(features)
scaled_data = pd.DataFrame(scaled_features, columns=features.columns, index=features.index)

# Add the target variable back
scaled_data['ERCOT'] = merged_data['ERCOT']

print("Preprocessed dataset is ready!")
print(scaled_data.head())

# Load ERCOT Electricity Load Data
data = pd.read_excel(r'D:\1\Load\DataSet\LoadTexasERCOT.xlsx')

# Convert timestamp column to datetime
data['Hour_End'] = pd.to_datetime(data['Hour_End'])
data.set_index('Hour_End', inplace=True)

# Extract Features
data['Hour'] = data.index.hour
data['Day'] = data.index.day
data['Month'] = data.index.month
data['Weekday'] = data.index.weekday

# Target Variable: ERCOT Load
ercot_values = data['ERCOT']

# Train-Test Split (2012-2014 for training, 2015 for testing)
train_data = ercot_values['2012':'2014']
test_data = ercot_values['2015']

# Normalize the data (Min-Max Scaling)
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data.values.reshape(-1, 1))
test_scaled = scaler.transform(test_data.values.reshape(-1, 1))

def create_sequences(data, seq_length):
    x, y = [], []
    for i in range(len(data) - seq_length):
        x.append(data[i:i + seq_length])  # Past sequence length data
        y.append(data[i + seq_length])    # Target value
    return np.array(x), np.array(y)

seq_length = 24  # Use past 24 hours to predict the next value
x_train, y_train = create_sequences(train_scaled, seq_length)
x_test, y_test = create_sequences(test_scaled, seq_length)

# Reshape inputs to fit GRU model
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

model = Sequential([
    GRU(100, activation='relu', return_sequences=True, input_shape=(seq_length, 1)),  # First GRU Layer
    Dropout(0.2),  
    GRU(50, activation='relu', return_sequences=False),  # Second GRU Layer
    Dropout(0.2),
    Dense(25, activation='relu'),  # Fully connected layer
    Dense(1)  # Output layer
])

# Compile Model
model.compile(optimizer='adam', loss='mse')
model.summary()

history = model.fit(x_train, y_train, epochs=20, batch_size=32, verbose=1, validation_data=(x_test, y_test))

# Make predictions
predictions_scaled = model.predict(x_test)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Reshape before inverse transform
predictions = scaler.inverse_transform(predictions_scaled.reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

# Compute error metrics
mae = mean_absolute_error(y_test_actual, predictions)
rmse = np.sqrt(mean_squared_error(y_test_actual, predictions))
mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual)) * 100
r2 = r2_score(y_test_actual, predictions)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"MAPE: {mape}%")
print(f"R² Score: {r2}")

plt.figure(figsize=(12, 6))
plt.plot(y_test_actual[:500], label='Actual', linewidth=2)
plt.plot(predictions[:500], label='Predicted', linestyle='dashed')
plt.legend()
plt.title('Actual vs Predicted Load (ERCOT)')
plt.xlabel('Time Steps')
plt.ylabel('Load (MW)')
plt.show()
# Calculate metrics
r2 = r2_score(y_test_actual, predictions)
mae = mean_absolute_error(y_test_actual, predictions)
rmse = np.sqrt(mean_squared_error(y_test_actual, predictions))
mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual)) * 100

# Print results
print(f"R²: {r2}")
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"MAPE: {mape}%")


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# Load ERCOT Electricity Load Data
ercot_data = pd.read_excel('D:/1/Load/DataSet/LoadTexasERCOT.xlsx')
ercot_data['Hour_End'] = pd.to_datetime(ercot_data['Hour_End'])
ercot_data.set_index('Hour_End', inplace=True)

# Load weather data
weather_dfs = []
for i in range(1, 4):
    df = pd.read_excel(f'D:/1/Load/DataSet/WeatherStation{i}.xlsx')
    df['Datetime'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour', 'Minute']])
    df.set_index('Datetime', inplace=True)
    weather_dfs.append(df)

# Merge all datasets
merged_data = ercot_data.copy()
for df in weather_dfs:
    merged_data = merged_data.merge(df, left_index=True, right_index=True, how='outer')

# Handle missing values
merged_data.fillna(method='ffill', inplace=True)
merged_data.fillna(method='bfill', inplace=True)

# Feature Engineering
merged_data['Hour'] = merged_data.index.hour
merged_data['Day'] = merged_data.index.day
merged_data['Month'] = merged_data.index.month
merged_data['Weekday'] = merged_data.index.weekday

# Normalization
scaler = MinMaxScaler()
features = merged_data.drop(columns=['ERCOT'])
scaled_features = scaler.fit_transform(features)
scaled_data = pd.DataFrame(scaled_features, columns=features.columns, index=features.index)
scaled_data['ERCOT'] = merged_data['ERCOT']

# Train-Test Split
train = scaled_data['2012':'2014']
test = scaled_data['2015']

# Sequence Creation
def create_sequences(data, seq_length):
    x, y = [], []
    for i in range(len(data) - seq_length):
        x.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(x), np.array(y)

seq_length = 24
x_train, y_train = create_sequences(train['ERCOT'].values, seq_length)
x_test, y_test = create_sequences(test['ERCOT'].values, seq_length)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

# Build Improved GRU Model
model = Sequential([
    GRU(128, return_sequences=True, activation='relu', input_shape=(seq_length, 1)),
    Dropout(0.2),
    BatchNormalization(),
    GRU(64, return_sequences=False, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
model.summary()

# Train Model
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Predictions
predictions_scaled = model.predict(x_test)
predictions = scaler.inverse_transform(predictions_scaled.reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

# Compute Metrics
mae = mean_absolute_error(y_test_actual, predictions)
rmse = np.sqrt(mean_squared_error(y_test_actual, predictions))
mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual)) * 100
r2 = r2_score(y_test_actual, predictions)

# Visualization
plt.figure(figsize=(12, 6))
plt.plot(y_test_actual[:500], label='Actual', linewidth=2)
plt.plot(predictions[:500], label='Predicted', linestyle='dashed')
plt.legend()
plt.title('Actual vs Predicted Load (ERCOT)')
plt.xlabel('Time Steps')
plt.ylabel('Load (MW)')
plt.show()

# Histogram of Errors
errors = y_test_actual - predictions
plt.figure(figsize=(10, 5))
sns.histplot(errors, bins=50, kde=True)
plt.title('Error Distribution')
plt.xlabel('Error')
plt.show()

# Print Results
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"MAPE: {mape}%")
print(f"R² Score: {r2}")


C:\Users\abc\AppData\Local\Temp\ipykernel_16128\3542382586.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.fillna(method='ffill', inplace=True)
C:\Users\abc\AppData\Local\Temp\ipykernel_16128\3542382586.py:31: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.fillna(method='bfill', inplace=True)


KeyError: '2015'